# Flask API

In [ ]:
from flask import Flask, request, jsonify
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer

# Paths to FAISS index and metadata
FAISS_INDEX_PATH = "/mnt/data/landmarks_municipalities_faiss.index"
METADATA_PATH = "/mnt/data/landmarks_municipalities_metadata.csv"

# Load the FAISS index
def load_faiss_index(index_path):
    return faiss.read_index(index_path)

# Load the metadata
def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

# Query the FAISS index
def query_faiss_index(index, metadata, query, model, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve metadata for the top results
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(metadata):
            result = {
                "rank": i + 1,
                "name": metadata.iloc[idx]["name"],
                "category": metadata.iloc[idx]["category"],
                "description": metadata.iloc[idx]["description"],
                "distance": float(distances[0][i])
            }
            results.append(result)

    return results

# Flask app initialization
app = Flask(__name__)

# Load resources globally for the app
print("Loading FAISS index and metadata...")
faiss_index = load_faiss_index(FAISS_INDEX_PATH)
metadata = load_metadata(METADATA_PATH)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Resources loaded successfully.")

# API endpoint for querying
@app.route('/query', methods=['POST'])
def query():
    data = request.json
    query_text = data.get("query", "")
    top_k = data.get("top_k", 5)

    if not query_text:
        return jsonify({"error": "Query text is required."}), 400

    try:
        # Perform the query
        results = query_faiss_index(faiss_index, metadata, query_text, embedding_model, top_k=top_k)
        return jsonify({"query": query_text, "results": results})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Main function to run the app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

## How to Run the Flask API
Install Dependencies: Ensure you have Flask and all required libraries installed:

In [ ]:
#!pip install flask pandas sentence-transformers faiss-cpu

2. Run the Script: Save the script as app.py and run it:

In [ ]:
# python app.py

3. Access the API: The API will be available at
```code
http://0.0.0.0:5000
```

# Query the API

In [ ]:
curl -X POST http://127.0.0.1:5000/query \
-H "Content-Type: application/json" \
-d '{"query": "beautiful beaches in Puerto Rico", "top_k": 5}'

### Response:

In [ ]:
{
    "query": "beautiful beaches in Puerto Rico",
    "results": [
        {
            "rank": 1,
            "name": "Flamenco Beach",
            "category": "landmark",
            "description": "Flamenco Beach is a public beach on the Caribbean island of Culebra...",
            "distance": 0.1234
        },
        {
            "rank": 2,
            "name": "Luquillo Beach",
            "category": "landmark",
            "description": "Luquillo Beach is a popular destination located on the northeastern coast of Puerto Rico...",
            "distance": 0.1345
        }
    ]
}

# Flask Code with Enhanced Features

In [ ]:
import logging
from flask import Flask, request, jsonify
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer

# Paths to FAISS index and metadata
FAISS_INDEX_PATH = "/mnt/data/landmarks_municipalities_faiss.index"
METADATA_PATH = "/mnt/data/landmarks_municipalities_metadata.csv"

# Logging setup
logging.basicConfig(filename="query_logs.log", level=logging.INFO, format="%(asctime)s - %(message)s")

# Load the FAISS index
def load_faiss_index(index_path):
    return faiss.read_index(index_path)

# Load the metadata
def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

# Query the FAISS index
# Modify the query_faiss_index function to include location data (latitude and longitude) if available
def query_faiss_index(index, metadata, query, model, top_k=5, filters=None):
    # Generate embedding for the query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve metadata for the top results
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(metadata):
            result = {
                "rank": i + 1,
                "name": metadata.iloc[idx]["name"],
                "category": metadata.iloc[idx]["category"],
                "description": metadata.iloc[idx]["description"],
                "distance": float(distances[0][i]),
                "latitude": metadata.iloc[idx].get("latitude", None),
                "longitude": metadata.iloc[idx].get("longitude", None)
            }

            # Apply filters
            if filters:
                for key, value in filters.items():
                    if result[key] != value:
                        break
                else:
                    results.append(result)
            else:
                results.append(result)

    return results

# Flask app initialization
app = Flask(__name__)

# Load resources globally for the app
print("Loading FAISS index and metadata...")
faiss_index = load_faiss_index(FAISS_INDEX_PATH)
metadata = load_metadata(METADATA_PATH)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Resources loaded successfully.")

# API endpoint for querying
@app.route('/query', methods=['POST'])
def query():
    data = request.json
    query_text = data.get("query", "")
    top_k = data.get("top_k", 5)
    filters = data.get("filters", {})

    if not query_text:
        return jsonify({"error": "Query text is required."}), 400

    try:
        # Perform the query
        results = query_faiss_index(faiss_index, metadata, query_text, embedding_model, top_k=top_k, filters=filters)

        # Log the query and results
        logging.info(f"Query: {query_text} | Filters: {filters} | Results: {len(results)}")

        return jsonify({"query": query_text, "results": results})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Main function to run the app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

### Explanation of Enhancements
1. Filters:
Added a filters field in the POST request. Filters allow users to refine results based on fields like category (e.g., "landmark").
Example POST body:
```code
{
    "query": "beaches",
    "top_k": 5,
    "filters": {
        "category": "landmark"
    }
}
```
2. Logging:
Logs query text, filters, and the number of results into query_logs.log for tracking and analytics.

3. Category Filtering Example:
If filters include {"category": "landmark"}, only landmarks are returned.

# Building the Frontend

We’ll create a simple and user-friendly web interface using HTML, Bootstrap, and jQuery. This will interact with the Flask API to display search results.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tourist Attraction Finder</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center">Tourist Attraction Finder</h1>
        <div class="mt-4">
            <form id="searchForm">
                <div class="form-group">
                    <label for="query">Enter your query:</label>
                    <input type="text" class="form-control" id="query" placeholder="e.g., beaches, historic sites" required>
                </div>
                <div class="form-group">
                    <label for="category">Category (Optional):</label>
                    <select class="form-control" id="category">
                        <option value="">All</option>
                        <option value="landmark">Landmark</option>
                        <option value="municipality">Municipality</option>
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Search</button>
            </form>
        </div>
        <div id="results" class="mt-5">
            <h3>Results</h3>
            <ul class="list-group" id="resultsList"></ul>
        </div>
    </div>

    <script>
        $(document).ready(function () {
            $("#searchForm").on("submit", function (event) {
                event.preventDefault();

                const query = $("#query").val();
                const category = $("#category").val();

                const data = {
                    query: query,
                    top_k: 5,
                    filters: {}
                };

                if (category) {
                    data.filters.category = category;
                }

                // Clear previous results
                $("#resultsList").empty();

                // Make AJAX request to Flask API
                $.ajax({
                    url: "http://127.0.0.1:5000/query",
                    type: "POST",
                    contentType: "application/json",
                    data: JSON.stringify(data),
                    success: function (response) {
                        const results = response.results;

                        if (results.length === 0) {
                            $("#resultsList").append("<li class='list-group-item'>No results found.</li>");
                        } else {
                            results.forEach((result) => {
                                $("#resultsList").append(`
                                    <li class="list-group-item">
                                        <h5>${result.name}</h5>
                                        <p>${result.description}</p>
                                        <small>Category: ${result.category}</small>
                                    </li>
                                `);
                            });
                        }
                    },
                    error: function () {
                        alert("Error querying the API.");
                    }
                });
            });
        });
    </script>
</body>
</html>

# Front-End with MAP

In [16]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tourist Attraction Finder with Map</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>
    <style>
        #map {
            height: 400px;
            width: 100%;
        }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center">Tourist Attraction Finder with Map</h1>
        <div class="mt-4">
            <form id="searchForm">
                <div class="form-group">
                    <label for="query">Enter your query:</label>
                    <input type="text" class="form-control" id="query" placeholder="e.g., beaches, historic sites" required>
                </div>
                <div class="form-group">
                    <label for="category">Category (Optional):</label>
                    <select class="form-control" id="category">
                        <option value="">All</option>
                        <option value="landmark">Landmark</option>
                        <option value="municipality">Municipality</option>
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Search</button>
            </form>
        </div>
        <div id="results" class="mt-5">
            <h3>Results</h3>
            <ul class="list-group" id="resultsList"></ul>
        </div>
        <div id="map" class="mt-4"></div>
    </div>

    <script>
        $(document).ready(function () {
            // Initialize the map
            const map = L.map('map').setView([18.2208, -66.5901], 8); // Centered on Puerto Rico

            // Add a tile layer (OpenStreetMap)
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
            }).addTo(map);

            // Form submission handler
            $("#searchForm").on("submit", function (event) {
                event.preventDefault();

                const query = $("#query").val();
                const category = $("#category").val();

                const data = {
                    query: query,
                    top_k: 10,
                    filters: {}
                };

                if (category) {
                    data.filters.category = category;
                }

                // Clear previous results and map markers
                $("#resultsList").empty();
                map.eachLayer((layer) => {
                    if (layer instanceof L.Marker) {
                        map.removeLayer(layer);
                    }
                });

                // Make AJAX request to Flask API
                $.ajax({
                    url: "http://127.0.0.1:5000/query",
                    type: "POST",
                    contentType: "application/json",
                    data: JSON.stringify(data),
                    success: function (response) {
                        const results = response.results;

                        if (results.length === 0) {
                            $("#resultsList").append("<li class='list-group-item'>No results found.</li>");
                        } else {
                            results.forEach((result) => {
                                // Display results in the list
                                $("#resultsList").append(`
                                    <li class="list-group-item">
                                        <h5>${result.name}</h5>
                                        <p>${result.description}</p>
                                        <small>Category: ${result.category}</small>
                                    </li>
                                `);

                                // Add marker to the map if coordinates are available
                                if (result.latitude && result.longitude) {
                                    const marker = L.marker([result.latitude, result.longitude]).addTo(map);
                                    marker.bindPopup(`<b>${result.name}</b><br>${result.description}`).openPopup();
                                }
                            });
                        }
                    },
                    error: function () {
                        alert("Error querying the API.");
                    }
                });
            });
        });
    </script>
</body>
</html>

IndentationError: unindent does not match any outer indentation level (<string>, line 15)

In [19]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tourist Attraction Finder with Marker Clustering</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
    <link rel="stylesheet" href="https://unpkg.com/leaflet.markercluster@1.5.3/dist/MarkerCluster.css" />
    <link rel="stylesheet" href="https://unpkg.com/leaflet.markercluster@1.5.3/dist/MarkerCluster.Default.css" />
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>
    <script src="https://unpkg.com/leaflet.markercluster@1.5.3/dist/leaflet.markercluster.js"></script>
    <style>
        #map {
            height: 400px;
            width: 100%;
        }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center">Tourist Attraction Finder with Marker Clustering</h1>
        <div class="mt-4">
            <form id="searchForm">
                <div class="form-group">
                    <label for="query">Enter your query:</label>
                    <input type="text" class="form-control" id="query" placeholder="e.g., beaches, historic sites" required>
                </div>
                <div class="form-group">
                    <label for="category">Category (Optional):</label>
                    <select class="form-control" id="category">
                        <option value="">All</option>
                        <option value="landmark">Landmark</option>
                        <option value="municipality">Municipality</option>
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Search</button>
            </form>
        </div>
        <div id="results" class="mt-5">
            <h3>Results</h3>
            <ul class="list-group" id="resultsList"></ul>
        </div>
        <div id="map" class="mt-4"></div>
    </div>

    <script>
        $(document).ready(function () {
            // Initialize the map
            const map = L.map('map').setView([18.2208, -66.5901], 8); // Centered on Puerto Rico

            // Add a tile layer (OpenStreetMap)
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
            }).addTo(map);

            // Initialize a marker cluster group
            const markerCluster = L.markerClusterGroup();

            // Form submission handler
            $("#searchForm").on("submit", function (event) {
                event.preventDefault();

                const query = $("#query").val();
                const category = $("#category").val();

                const data = {
                    query: query,
                    top_k: 10,
                    filters: {}
                };

                if (category) {
                    data.filters.category = category;
                }

                // Clear previous results and map markers
                $("#resultsList").empty();
                markerCluster.clearLayers();

                // Make AJAX request to Flask API
                $.ajax({
                    url: "http://127.0.0.1:5000/query",
                    type: "POST",
                    contentType: "application/json",
                    data: JSON.stringify(data),
                    success: function (response) {
                        const results = response.results;

                        if (results.length === 0) {
                            $("#resultsList").append("<li class='list-group-item'>No results found.</li>");
                        } else {
                            results.forEach((result) => {
                                // Display results in the list
                                $("#resultsList").append(`
                                    <li class="list-group-item">
                                        <h5>${result.name}</h5>
                                        <p>${result.description}</p>
                                        <small>Category: ${result.category}</small>
                                    </li>
                                `);

                                // Add marker to the cluster if coordinates are available
                                if (result.latitude && result.longitude) {
                                    const marker = L.marker([result.latitude, result.longitude])
                                        .bindPopup(`<b>${result.name}</b><br>${result.description}`);
                                    markerCluster.addLayer(marker);
                                }
                            });

                            // Add the cluster group to the map
                            map.addLayer(markerCluster);
                        }
                    },
                    error: function () {
                        alert("Error querying the API.");
                    }
                });
            });
        });
    </script>
</body>
</html>

IndentationError: unindent does not match any outer indentation level (<string>, line 18)